In [1]:
#误差反向传播法：高效计算权重参数的梯度方法
#复合函数的导数，链式法则，是一种累乘
#加法节点：z=x+y （加法的反向传播：上一级值等于本级值）
#乘法节点：z=x*y（乘法的反向传播：上一级输入信号*本级翻转值）

#简单层的实现
#乘法层：计算图的乘法节点
#加法层：计算图的加法节点

#层：神经网络中功能的单位。神经网络中的层实现为一个类。负责sigmoid函数的sigmoid，以层来实现
#层的实现有两个共通的方法（接口）：forward()//正向传播 backward()//反向传播

In [8]:
#乘法层
class MulLayer:
    def __init__(self):#初始化实例变量x与y，保存正向传播时的输入值
        self.x=None
        self.y=None
    def forward(self,x,y):
        self.x=x
        self.y=y
        out=x*y
        return out
    def backward(self,dout):#从上游传来的导数（dout）乘以正向传播的翻转值，传给下游
        dx=dout*self.y#翻转x和y
        dy=dout*self.x
        return dx,dy
    
apple=100
apple_num=2
tax=1.1
#layer
mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()
#forward
apple_price=mul_apple_layer.forward(apple,apple_num)
price=mul_tax_layer.forward(apple_price,tax)
print(price)#220.00000000000003
#backward
dprice=1
dapple_price,dtax=mul_tax_layer.backward(dprice)
print(dapple_price)#1.1
dapple,dapple_num=mul_apple_layer.backward(dapple_price)
print(dapple,dapple_num,dtax)

220.00000000000003
1.1
2.2 110.00000000000001 200


In [13]:
#加法层
class AddLayer:
    def __init__(self):
        pass
    def forward(self,x,y):
        out=x+y
        return out
    def backward(self,dout):
        dx=dout*1
        dy=dout*1
        return dx,dy


In [14]:
#实现2个苹果和3个橘子计算图
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1
#layer
mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()
#forward
apple_price=mul_apple_layer.forward(apple,apple_num)
orange_price=mul_orange_layer.forward(orange,orange_num)
all_price=add_apple_orange_layer.forward(apple_price,orange_price)
price=mul_tax_layer.forward(all_price,tax)
#backward
dprice=1
dall_price,dtax=mul_tax_layer.backward(dprice)#其实发现参数上下对应，就是函数顺序颠倒了
dapple_price,dorange_price=add_apple_orange_layer.backward(dall_price)
dorange,dorange_num=mul_orange_layer.backward(dorange_price)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)



In [15]:
print(price)

715.0000000000001


In [16]:
print(dapple_num,dapple,dorange,dorange_num,dtax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


In [17]:
#将计算图的思路应用到神经网络中
#ReLu层（正向传播时输入x大于0，，正向传播值为本身，反向传播时（求导）将上游值原封不动传给下游；x小于等于0，反向传播时值为0，上游信号停在此处）
#forward 与backward是numpy数组
class Relu:
    def __init__(self):
        self.mask=None
    def forward(self,x):
        self.mask=(x<=0)#mask数组是bool型
        out=x.copy()
        out[self.mask]=0
        return out
    def backward(self,dout):
        dout[self.mask]=0
        dx=dout
        return dx

In [21]:
import numpy as np
x=np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [22]:
mask=(x<=0)
print(mask)

[[False  True]
 [ True False]]


In [24]:
#sigmoid层
#多了“/节点”和“exp”节点
#但简化后，只和正向传播的输出和上游值有关
class sigmoid:
    def __init__(self):
        self.out=None
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out
        return out
    def backward(self,dout):
        dx=dout*self.out*(1.0-self.out)
        return dx

In [25]:
#Affine/Softmax层:np.dot(x,w)+b的正向反向传播
class Affine:
    def __init__(self,W,b):
        self.W=W#需要不同函数之间使用时，才self.xx创建变量，当变量需要外部输入时，才传入函数参数
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
    def forward(self,x):
        self.x=x
        out=np,dot(x,self.W)+self.b
        return out
    def backward(self,dout):
        dx=np.dot(dout,self.W.T)
        self.dW=np.dot(self.x.T,dout)
        self.db=np.sum(dout,axis=0)#多批量输入数据
        return dx
        


In [26]:
#输入图像通过Affine层和ReLu层进行转换（一层Affine一层ReLu一层Affine一层ReLu...最后一层Affine），最后一层Affine再通过softmax层进行正规化
#最后一层Affine输出10个数据，数值为得分，得分高的所在位置为对应类别。softmax输入为将这10个数据，输出为10个数据，输出是概率值（0<值<1）
#10分类因为这个例子是数字识别

#神经网络中进行的处理有推理和学习。
#推理时,只需要给出答案，所以只对得分最大值感兴趣，会将最后一个Affine层的输出作为识别结果（神经网络中未被正规化的输出结果称为“得分”）
#学习时，需要softmax层



In [27]:
#Softmax-with-Loss层
#包含softmax层以及作为损失函数的交叉熵误差

#正向传播：假设要进行3分类，从前面的层接收3个输入（得分）。softmax层将输入（a1,a2,a3）正规化，输出（y1,y2,y3）。
#cross entropy error层接收softmax的输出（y1,y2,y3）和监督标签(t1,t2,t3),从这些数据中输出损失L

#（y1,y2,y3）是softmax层的输出
#（t1,t2,t3）是监督数据
#softmax层的反向传播输出得到了（y1-t1,y2-t2,y3-t3）的结果，是softmax层的输出与监督数据的差分
#神经网络的反向传播会把这个差分表示的误差传递给前面的层，这是神经网络学习的重要性质

#神经网络学习的目的就是通过调整权重参数，使神经网络的输出接近监督标签
#因此必须将神经网络的输出与监督标签的误差高效的传给前面的层：这里就是差分
#举例：监督标签（0,1,0），softmax层输出（0.3,0.2，0.5），正确解标签处的概率是0.2（20%），这时候神经网络未能正确识别。softmax层反向传递
#（0.3,-0.8,0.5）这样一个大的误差，大的误差向前面的层传播，神经网络会学习到“大”的内容

#为何Softmax-with-Loss层反向传播结果是（y1-t1,y2-t2,y3-t3）
#因为：使用交叉熵误差函数作为softmax函数的损失函数
#使用平方和误差函数作为恒等函数（输入等于输出）的误差函数，也可以得到（y1-t1,y2-t2,y3-t3）

class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None#损失
        self.y=None#损失函数的输出
        self.t=None#监督数据one-hot vector
    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size#除以批的大小，传给前面的层是单个数据的误差
        return dx
        
        

In [42]:
import sys,os
sys.path.append(os.getcwd())
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict#y有序字典，会记住向字典里添加的元素的顺序
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,putput_size,weight_init_std=0.01):
        #初始化权重
        self.params={}
        self.params["W1"]=weight_init_std*np.random.randn(input_size,hidden_size)#索引为“W1”的是数组
        self.params["b1"]=np.zeros(hidden_size)
        self.params["W2"]=weight_init_std*np.random.randn(hidden_size,output_size)
        self.params["b2"]=np.zeros(output_size)
        #生成层(叠乐高一样一层一层往上叠)
        self.layers=OrderedDict()
        self.layers["Affine1"]=Affine(self.params["W1"],self.params["b1"])
        self.layers["Relu1"]=Relu()
        self.layers["Affine2"]=Affine(self.params["W2"],self.params["b2"])
        self.lastlayer=SoftmaxWithLoss()
    def predict(self,x):
        for layer in self.layers.values():#self.layers.values()，组成元素是层，key是索引“Affine”
            x=layer.forward(x)
        return x
    def loss(self,x,t):
        y=self.predict(x)
        return self.lastlayer.forward(y,t)
    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        if t.ndim!=1:
            t=np.argmax(t,axis=1)#one-hot
        accuracy=np.sum(y==t)/float(x.shape[0])
        return accuracy
    def numerical_gradient(self,x,t):#数值微分求梯度
        loss_W=lambda W:self.loss(x,t)
        grads={}
        grads["W1"]=numerical_gradient(loss_W,self.params["W1"])#函数名一样但是参数个数不一样
        grads["b1"]=numerical_gradient(loss_W,self.params["b1"])
        grads["W2"]=numerical_gradient(loss_W,self.params["W2"])
        grads["b2"]=numerical_gradient(loss_W,self.params["b2"])
        return grads
    def gradient(self,x,t):#反向传输求梯度
        #forward
        self.loss(x,t)
        #backward
        dout=1
        dout=self.lastLayer.backward(dout)
        layers=list(self.layers.values())#list1 = ['physics', 'chemistry', 1997, 2000] print "list1[0]: ", list1[0] //list1[0]:  physics
        layers.reverse()##list.reverse()，对列表元素进行反向排列
        for layer in layers:
            dout=layer.backward(dout)
        #设定
        grads={}
        grads["W1"]=self.layers["Affines1"].dW
        grads["b1"]=self.layers["Affines1"].db
        grads["W2"]=self.layers["Affines2"].dW
        grads["b2"]=self.layers["Affines2"].db
        return grads
    
  
        


In [31]:
x=np.array([[1,2],[3,4]])
for idx, x in enumerate(x):
    print(idx)
    print(x)

0
[1 2]
1
[3 4]


In [39]:
 #self.layers.values()  //layers的元素
#dishes = {'eggs': 2, 'sausage': 1, 'bacon': 1, 'spam': 500}
#keys = dishes.keys()
#values = dishes.values()
#>>> # 迭代
#>>> n = 0
#>>> for val in values:
#...     n += val
#>>> print(n)
#504

#>>> # keys 和 values 以相同顺序（插入顺序）进行迭代
#>>> list(keys)     # 使用 list() 转换为列表
#['eggs', 'sausage', 'bacon', 'spam']
#>>> list(values)
#[2, 1, 1, 500]


In [43]:
#以上实现代码：
#将神经网络的层保存为OrderedDict这一点非常重要。OrderDict是有序字典，“有序”是指它可以记住向字典里添加元素的顺序
#神经网络正向传播只需按照添加元素的顺序，调用各层的forward()方法就可以完成处理。而反向传播只需要按照相反的顺序调用各层。

#因为Affine层和ReLU层的内部会正确处理正向传播和反向传播，所以这里仅仅是以正确的顺序连接各层，再按顺序（或者逆序）调用各层

#像这样通过将神经网络的组成元素以层的方式实现，可以轻松的构建神经网络。
#用层进行模块化的实现具有很大优势。因为如果想另外构建一个神经网络（N层），只需要像组装乐高积木那样添加必要层就可以了。之后，
#通过各层内部实现的正向传播和反向传播，就可正确计算进行识别以及所需梯度

In [48]:
#误差反向传播法梯度确认
#数值微分不易出错，反向传播容易出错。需要进行梯度确认：确认数值微分求出的梯度结果和误差反向传播算法求出的结果是否一致
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
#读入数据
(x_train,t_train),(x_test,t_test)=load_mnist(normalize=True,one_hot_label=True)
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
x_batch=x_train[:3]
t_batch=t_train[:3]
grad_numerical=network.numerical_gradient(x_batch,t_batch)
grad_backprop=network.gradient(x_batch,t_batch)
#print(grad_numerical.keys())
#求各个权重的绝对误差的平均值
for key in grad_numerical.keys():
    print(grad_backprop[key])
    print(grad_numerical[key])
    diff=np.average(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key+":"+str(diff))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
W1:2.162681404935406e-10
[ 1.58157529e-03  2.08517996e-03  3.91271147e-04 -1.73121549e-04
  5.01897002e-04  2.68009589e-04  2.63441547e-04  6.00789606e-04
  1.78196722e-04  1.91918033e-03 -7.99633878e-04  2.66332198e-04
 -3.28289378e-04  4.27201522e-05  2.37272700e-04  1.81698056e-03
  2.52448820e-04 -8.97545318e-04 -4.21030426e-04 -4.71887234e-04
 -7.53862402e-04 -9.03932395e-05 -1.30732735e-03 -1.04167895e-03
  1.51972141e-03  2.59372005e-03 -1.15084497e-04  1.33178448e-03
  1.80002035e-03 -1.48027231e-05  9.78016878e-04  3.43179233e-03
  1.10263052e-03  1.18863214e-03 -6.50195491e-04 -4.74212040e-04
  6.15965442e-04 -4.18172012e-04  1.48891297e-04 -2.74121709e-06
  1.01068221e

In [49]:
#利用误差反向传播算法的学习
(x_train,t_train),(x_test,t_test)=load_mnist(normalize=True,one_hot_label=True)
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
iters_num=10000
train_size=x_train.shape[0]
batch_size=100
learning_rate=0.1
train_loss_list=[]
train_acc_list=[]
test_acc_list=[]
iter_per_epoch=max(train_size/batch_size,1)
for i in range(iters_num):
    batch_mask=np.random.choice(train_size,batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    #通过误差反向传播法求梯度
    grad=network.gradient(x_batch,t_batch)
    #更新
    for key in ("W1","b1","W2","b2"):
        network.params[key]-=learning_rate*grad[key]
        loss=network.loss(x_batch,t_batch)
        train_loss_list.append(loss)
        if i%iter_per_epoch==0:
            train_acc=network.accuracy(x_train,t_train)
            test_acc=network.accuracy(x_test,t_test)
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            print(train_acc,test_acc)

0.0971 0.0979
0.0971 0.0979
0.09736666666666667 0.0982
0.09736666666666667 0.0982
0.78615 0.7927
0.7861666666666667 0.7928
0.7866166666666666 0.7936
0.78675 0.7937
0.8783166666666666 0.883
0.8783333333333333 0.8829
0.87905 0.8836
0.8790333333333333 0.8836
0.8986833333333333 0.9003
0.8986 0.9004
0.8985166666666666 0.9006
0.8985666666666666 0.9011
0.9069 0.9093
0.9069166666666667 0.9093
0.907 0.9096
0.90695 0.9094
0.9127666666666666 0.9151
0.91275 0.915
0.9130333333333334 0.9145
0.9130333333333334 0.9145
0.9178666666666667 0.9205
0.9178833333333334 0.9206
0.9178333333333333 0.9203
0.9178333333333333 0.9201
0.9216333333333333 0.9237
0.9216666666666666 0.9236
0.92175 0.9227
0.9217166666666666 0.9227
0.9257833333333333 0.9278
0.9257833333333333 0.9278
0.9259333333333334 0.9278
0.92595 0.9278
0.9294 0.9308
0.9293833333333333 0.9308
0.9294166666666667 0.9309
0.9294166666666667 0.9309
0.9326 0.9337
0.9326 0.9338
0.93265 0.9341
0.9326333333333333 0.9343
0.9353 0.9369
0.9352666666666667 0.9369
0

KeyboardInterrupt: 